# $\Lambda_b \rightarrow pK\mu\tau$ documentation: momentum reconstruction

There are two main parts to our MSci project:
1. Reconstructing the tau momentum
2. Reducing the background

In this notebook I will be talking about the tau momentum reconstruction and in the following I will be talking about background reduction.

## Tau momentum reconstruction
The tau momentum was obtained by:
1. Finding the $pK\mu$ transverse momentum relative to the $\Lambda_b$ line of flight. The $\tau$ transverse momentum is then the opposite of that value
2. Finding the tau decay point, which is the intersection between the tauMu line of flight and the plane defined by the $pK\mu$ momentum and the $\Lambda_b$ line of flight
3. The $\tau$ momentum can then be calculated with the magnitude of its transverse momentum and the direction of the $\tau$ flight

Below I will be outlining the main functions used to obtain the tau momentum

## Loading the data
The data can be loaded by calling the `load data` function. 
Three types of data sets can be obtained: real data (`Lb_data`), B Monte Carlo data (`B_MC`) and $\Lambda_b$ Monte Carlo data (`Lb_MC`).
We will proceed to clean the data before doing heavy calculations to reduce overall computing time.
If you want to see what the data looks like uncleaned, just comment out the `if-elif-else` statements in the cell below.


In [1]:
from background_reduction.b_MC_reduction import b_cleaning
from background_reduction.data_reduction import reduce_background
from data.data_loader import load_data
df_name = 'Lb_data'

data_frame = load_data(df_name=df_name)
if df_name == 'Lb_data':
    data_frame = reduce_background(data_frame, True)
elif df_name == 'B_MC':
    data_frame = b_cleaning(data_frame)
else:
    data_frame = data_frame

RangeIndex(start=0, stop=294631, step=1)


## Finding the tau  decay point
The tau decay point was obtained by 
1. Defining a plane formed from the $\Lambda_b$ line of flight and the $pK\mu$ momentum
2. Defining a line of flight for tauMu. This line of flight is obtained from the tauMu reference point and from its momentum
3. Finding the intersection between tauMu and the plane. This intersection corresponds to the $\tau$ decay point and helps us in determining the $\tau$ line of flight.

In [ ]:
from get_vertex import obtain_lb_line_of_flight, line_plane_intersection
data_frame = obtain_lb_line_of_flight(data_frame)
data_frame = line_plane_intersection(data_frame)

data frame length: 294631, FD 90th percentile: 37.37454094024642


## Finding the tau momentum
The tau momentum could then be obtained by
1. Finding the transverse momentum of the $pK\mu$ particles. The transverse momentum of the $\tau$ will then just be the opposite of that value
2. Finding the angle between the $\tau$ line of flight and the $\Lambda_b$ line of flight
3. Obtaining the tau momentum from those values.

In [ ]:
from get_vertex import transverse_momentum, tau_momentum_mass
data_frame = transverse_momentum(data_frame)
data_frame = tau_momentum_mass(data_frame)

## Mass plots
We can then use the previous data to plot the reconstructed $pK\mu\tau$ mass, as well as the $pK\mu\mu$ mass. 

In [ ]:
from masses import get_mass
import matplotlib.pyplot as plt
particles_associations = [['Kminus_P', 'K'], ['proton_P', 'proton'], ['mu1_P', 'mu'], ['tau_P', 'tau']]
data_frame['pkmutau_mass'] = get_mass(data_frame=data_frame, particles_associations=particles_associations)
particles_associations = [['Kminus_P', 'K'], ['proton_P', 'proton'], ['mu1_P', 'mu'], ['tauMu_P', 'mu']]
data_frame['pkmumu_mass'] = get_mass(data_frame=data_frame, particles_associations=particles_associations)

plt.figure(figsize=(3.5, 3), dpi=300)
plt.hist(data_frame[(data_frame['pkmumu_mass'] < 5620 - 40) | (data_frame['pkmumu_mass'] > 5620 + 40)]['pkmutau_mass'],
        bins=100, range=[4000, 30000])
plt.xlabel('$m_{pK\\mu\\tau}$')
plt.tight_layout()
plt.show()
plt.figure(figsize=(3.5, 3), dpi=300)
plt.hist(data_frame['pkmumu_mass'], bins=100, range=[4000, 9000])
plt.xlabel('$m_{pK\\mu\\mu}$')
plt.tight_layout()
plt.show()